In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from langsmith import traceable
from utils import get_vector_db_retriever
from groq import Groq
from typing import List
import nest_asyncio

MODEL_PROVIDER = "groq"
MODEL_NAME = "openai/gpt-oss-20b"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

groq_client = Groq()
nest_asyncio.apply()

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_groq(messages)


retriever = get_vector_db_retriever()


@traceable
def retrieve_documents(question: str):
    """Fetches documents from the vector store based on the question."""
    return retriever.invoke(question)

@traceable(
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_groq(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return groq_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )


@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

@traceable
def chat_bot():
    print("This is a simple chatbot. Type 'bye bye' to quit.\n")
    
    while True:
        ques = input("What's your question: ")
        print(f"Question: {ques}")
        
        if ques.lower() == 'bye bye':
            print("au revoir")
            break
        
        response = langsmith_rag(ques)
        print(f"AI: {response}\n")


chat_bot()

This is a simple chatbot. Type 'bye bye' to quit.

Question: whos batman
AI: Batman is the vigilante superhero created by DC Comics, whose alter‑ego is Bruce Wayne, a wealthy industrialist from Gotham City. He fights crime using his intellect, detective skills, and an array of gadgets, often aided by allies like Robin and Commissioner Gordon. The character has become an iconic figure in comic book and popular culture worldwide.

Question: whos chacha chaudhary
AI: I’m sorry, but I don’t have that information.

Question: first president of india?
AI: The first President of India was Dr Rajendra Prasad. He served from 1950, when India became a republic, until 1962. His tenure laid the groundwork for the country's early constitutional governance.

Question: whos chacha chaudhary
AI: Chacha Chaudhary is a popular Indian comic‑book hero created by Pran Kumar Sharma. He’s a clever, super‑intelligent old man who solves problems with a single “brain” punch. The character first appeared in 1971

In [ ]:
from langsmith import traceable
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever
from langchain_groq import ChatGroq  # Groq LLM wrapper

MODEL_PROVIDER = "groq"
MODEL_NAME = "llama-3.1-8b-instant"   # free fast Groq model
APP_VERSION = 1.0

RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

# Groq client
groq_client = ChatGroq(model=MODEL_NAME, temperature=0.0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_groq` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"}
    ]
    return call_groq(messages)

"""
call_groq
- Returns the chat completion output from Groq
"""
@traceable(run_type="llm")
def call_groq(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    # ChatGroq uses LCEL invoke, not openai-style
    response = groq_client.invoke(messages)
    return response.content  # returns plain string

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response


USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\github_projects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
question = "How does LangSmith help developers with their applications?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"website": "www.google.com"}})
print(ai_answer)